<a href="https://colab.research.google.com/github/Ditsuhi/Nitrogen_Dioxide_Prediction/blob/main/GridSearchCV_BlockingTimeSeriesSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In the following two cells are the code for parameter optimisation.
# First of all, the class was created in order to split data.

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


btscv = BlockingTimeSeriesSplit(n_splits=3)

#create_model is the model architecture
model = KerasRegressor(build_fn=create_model, verbose=0)


#define the grid search parameters


optimizer = ['RMSprop',  'Adam']
kernel_size = [(3, 3), (5, 5), (7, 7), (9, 9)]
filters= [8, 16, 32]
merge_mode=['sum', 'concat', 'ave', 'mul']
number of Layers = [2, 3, 4]


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
param_grid = dict(filters=filters,  kernel_size=kernel_size, optimizer=optimizer, merge_mode= merge_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=btscv)
grid_result = grid.fit(X_train_reshaped, y_train_reshaped, epochs = 20)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))